# **Quantum Randomness Meets AI: Building a Quantum-Enhanced GAN**

*Unlock the power of Quantum Random Number Generators (QRNGs) to enhance your Generative Adversarial Networks (GANs) and take synthetic data generation to the next level.*

---

## **Introduction**

As artificial intelligence (AI) continues to evolve, the integration of quantum computing concepts offers exciting possibilities. One such integration is the use of **Quantum Random Number Generators (QRNGs)** in AI models. QRNGs leverage the inherent unpredictability of quantum mechanics to produce truly random numbers, unlike classical pseudo-random number generators (PRNGs), which are deterministic and potentially predictable.

In this tutorial, we'll explore how to incorporate QRNGs into a Generative Adversarial Network (GAN) to generate synthetic images resembling the MNIST handwritten digits. We'll cover:

- Setting up the QRNG with a caching mechanism.
- Building a Quantum Noise Generator using QRNG.
- Implementing a GAN that uses quantum noise in the generation process.
- Training the Quantum GAN and visualizing the results.

By the end, you'll have a working Quantum GAN that leverages quantum randomness to potentially enhance the diversity and unpredictability of the generated data.

---

## **Why Quantum Randomness in GANs?**

### **Limitations of Pseudo-Randomness**

- **Predictability**: PRNGs are algorithmically generated and can be predicted if the initial seed is known.
- **Limited Entropy**: PRNGs may not provide sufficient randomness for certain applications requiring high entropy.

### **Advantages of Quantum Randomness**

- **True Randomness**: QRNGs produce numbers based on quantum phenomena, ensuring unpredictability.
- **Enhanced Diversity**: Incorporating quantum randomness may introduce more variation in generated data.
- **Security Benefits**: True randomness can improve the robustness of models against certain types of attacks.

---

## **Overview**
Below is the flowchart representing the approach:
<img src="tutorial4-flowchart.png" alt="Flowchart" width=auto />

---
## **Prerequisites**

- **Python 3.6+**
- **PyTorch**
- **NumPy**
- **Requests library**
- **Torchvision**
- **An API token for your Quantum eMotion's Entropy-as-a-Service (EaaS) API**

In [ ]:
import numpy as np
import requests
import base64
from datetime import datetime, timedelta
from functools import lru_cache
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from typing import Tuple


## **1. Setting Up the Quantum Random Number Generator**

We'll create a `QuantumRandomGenerator` class to interact with the QRNG API. This class includes a caching mechanism to minimize API calls.

In [ ]:
class QuantumRandomGenerator:
    def __init__(self, api_token: str, cache_ttl_minutes: int = 30):  # Fixed __init__ method
        self.api_token = api_token
        self.base_url = 'https://your-qrng-api-endpoint/entropy'
        self.headers = {'Authorization': f'Bearer {self.api_token}'}
        self.cache_ttl = timedelta(minutes=cache_ttl_minutes)
        self._initialize_cache()

    def _initialize_cache(self):
        @lru_cache(maxsize=32)
        def cached_quantum_fetch(num_bytes: int, timestamp: str) -> np.ndarray:
            response = requests.get(
                self.base_url,
                headers=self.headers,
                params={'size': num_bytes},
                timeout=10
            )
            response.raise_for_status()
            data = response.json()
            qrng_base64 = data['random_number']
            qrng_bytes = base64.b64decode(qrng_base64)
            return np.frombuffer(qrng_bytes, dtype=np.uint8)
        self._cached_fetch = cached_quantum_fetch

    def _get_cache_key_timestamp(self) -> str:
        now = datetime.now()
        ttl_seconds = self.cache_ttl.total_seconds()
        epoch_seconds = now.timestamp()
        current_period = int(epoch_seconds // ttl_seconds)
        period_start = datetime.fromtimestamp(current_period * ttl_seconds)
        return period_start.isoformat()

    def get_quantum_random(self, num_bytes: int) -> np.ndarray:
        if num_bytes > 512:
            raise ValueError("num_bytes cannot exceed 512")
        cache_timestamp = self._get_cache_key_timestamp()
        result = self._cached_fetch(num_bytes, cache_timestamp)
        if result is not None:
            return result
        else:
            raise Exception("Failed to fetch quantum random numbers")

    def clear_cache(self):
        self._cached_fetch.cache_clear()

**Important Notes:**

- **API Endpoint**: Replace `'https://your-qrng-api-endpoint/entropy'` with your actual QRNG API endpoint.
- **API Token**: Ensure you have a valid API token and set it appropriately.
- **Caching Mechanism**: Uses an LRU cache to minimize API calls and handle rate limits.

---

## **2. Managing Quantum Random Numbers Efficiently**

We'll implement a buffer system with the `QuantumRandomBuffer` class to handle large requests and minimize API calls.

In [ ]:
class QuantumRandomBuffer:
    VALID_SIZES = [4, 8, 16, 32, 64, 128, 256, 512]

    def __init__(self, qrng: QuantumRandomGenerator):  # Fixed __init__ method
        self.qrng = qrng
        self.buffer = np.array([], dtype=np.uint8)

    def _get_optimal_chunk_size(self, required_size: int) -> int:
        for size in self.VALID_SIZES:
            if size >= required_size:
                return size
        return self.VALID_SIZES[-1]

    def get_numbers(self, size: int) -> np.ndarray:
        while len(self.buffer) < size:
            remaining = size - len(self.buffer)
            chunk_size = self._get_optimal_chunk_size(min(512, remaining))
            new_numbers = self.qrng.get_quantum_random(chunk_size)
            self.buffer = np.concatenate([self.buffer, new_numbers])
        result = self.buffer[:size]
        self.buffer = self.buffer[size:]
        return result

**Key Features:**

- **Buffering**: Stores quantum random numbers to reduce API requests.
- **Optimal Chunk Size**: Ensures requests are made in sizes supported by the API.

---

## **3. Implementing Quantum Noise for the GAN**

We'll create a `QuantumNoiseGenerator` class that uses the `QuantumRandomBuffer` to generate noise for the GAN's latent space.

In [ ]:
class QuantumNoiseGenerator:
    def __init__(self, qrng_buffer: QuantumRandomBuffer):
        self.qrng_buffer = qrng_buffer
        
    def generate_noise(self, batch_size: int, latent_dim: int) -> torch.Tensor:
        # Get quantum random numbers and convert to normal distribution
        size = batch_size * latent_dim
        quantum_bytes = self.qrng_buffer.get_numbers(size)
        
        # Convert to uniform [0, 1]
        uniform = quantum_bytes.astype(np.float32) / 255.0
        
        # Convert to normal distribution using Box-Muller transform
        shape = (batch_size, latent_dim)
        if size % 2 != 0:
            uniform = np.append(uniform, self.qrng_buffer.get_numbers(1)[0] / 255.0)
            
        uniform_reshape = uniform.reshape(-1, 2)
        
        r = np.sqrt(-2.0 * np.log(uniform_reshape[:, 0]))
        theta = 2.0 * np.pi * uniform_reshape[:, 1]
        
        normal = np.column_stack([r * np.cos(theta), r * np.sin(theta)])
        normal = normal.reshape(shape)[:batch_size, :latent_dim]
        
        return torch.from_numpy(normal).float()

**Explanation:**

- **Uniform to Normal**: Uses the Box-Muller transform to convert uniformly distributed quantum random numbers into normally distributed noise suitable for the GAN's latent space.
- **Batch Processing**: Generates noise in batches to match the input requirements of the GAN.

---

## **4. Building the Quantum GAN**

### **4.1 The Generator**

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim: int):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, 784),
            nn.Tanh()
        )
        
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        img = self.model(z)
        return img.view(-1, 1, 28, 28)

### **4.2 The Discriminator**

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img: torch.Tensor) -> torch.Tensor:
        img_flat = img.view(-1, 784)
        return self.model(img_flat)

### **4.3 The Quantum GAN Class**

In [ ]:
class QuantumGAN:
    def __init__(self, 
                 latent_dim: int,
                 qrng_buffer: QuantumRandomBuffer,
                 device: str = "cuda" if torch.cuda.is_available() else "cpu"):
        self.latent_dim = latent_dim
        self.device = device
        self.quantum_noise = QuantumNoiseGenerator(qrng_buffer)
        
        self.generator = Generator(latent_dim).to(device)
        self.discriminator = Discriminator().to(device)
        
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        
        self.criterion = nn.BCELoss()
        
    def train_step(self, real_images: torch.Tensor) -> Tuple[float, float]:
        batch_size = real_images.size(0)
        real_label = torch.ones(batch_size, 1).to(self.device)
        fake_label = torch.zeros(batch_size, 1).to(self.device)
        
        # Train Discriminator
        self.d_optimizer.zero_grad()
        
        real_output = self.discriminator(real_images)
        d_loss_real = self.criterion(real_output, real_label)
        
        z = self.quantum_noise.generate_noise(batch_size, self.latent_dim).to(self.device)
        fake_images = self.generator(z)
        fake_output = self.discriminator(fake_images.detach())
        d_loss_fake = self.criterion(fake_output, fake_label)
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        self.d_optimizer.step()
        
        # Train Generator
        self.g_optimizer.zero_grad()
        
        z = self.quantum_noise.generate_noise(batch_size, self.latent_dim).to(self.device)
        fake_images = self.generator(z)
        fake_output = self.discriminator(fake_images)
        
        g_loss = self.criterion(fake_output, real_label)
        g_loss.backward()
        self.g_optimizer.step()
        
        return d_loss.item(), g_loss.item()

**Key Components:**

- **Quantum Noise in Latent Space**: Uses quantum-generated noise for the generator's input.
- **GAN Training Loop**: Standard GAN training steps adapted to include quantum noise.
- **Loss Functions**: Uses Binary Cross-Entropy Loss for both generator and discriminator.

---

## **5. Training the Quantum GAN**

### **5.1 Training Function**

In [ ]:
def train_quantum_gan(qrng_token: str, 
                     num_epochs: int = 100,
                     batch_size: int = 64,
                     latent_dim: int = 100,
                     save_dir: str = "generated_images"):
    # Create save directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    # Initialize QRNG
    qrng = QuantumRandomGenerator(api_token=qrng_token)
    qrng_buffer = QuantumRandomBuffer(qrng)
    
    # Setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load MNIST dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    mnist = torchvision.datasets.MNIST(
        root='./data',
        train=True,
        transform=transform,
        download=True
    )
    
    dataloader = DataLoader(mnist, batch_size=batch_size, shuffle=True)
    
    # Initialize Quantum GAN
    qgan = QuantumGAN(latent_dim=latent_dim, qrng_buffer=qrng_buffer, device=device)
    
    # Training loop
    for epoch in range(num_epochs):
        for batch_idx, (real_images, _) in enumerate(dataloader):
            real_images = real_images.to(device)
            d_loss, g_loss = qgan.train_step(real_images)
            
            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] "
                      f"Batch [{batch_idx}/{len(dataloader)}] "
                      f"D_loss: {d_loss:.4f} "
                      f"G_loss: {g_loss:.4f}")
                
                # Save generated images
                with torch.no_grad():
                    z = qgan.quantum_noise.generate_noise(16, latent_dim).to(device)
                    fake_images = qgan.generator(z)
                    torchvision.utils.save_image(
                        fake_images,
                        os.path.join(save_dir, f'fake_images_epoch_{epoch}_batch_{batch_idx}.png'),
                        normalize=True,
                        nrow=4
                    )
    
    return qgan

**Training Details:**

- **Dataset**: Uses the MNIST dataset of handwritten digits.
- **Checkpointing**: Saves generated images every 100 batches for visualization.
- **Epochs and Batch Size**: Configurable parameters to control training duration.

### **5.2 Running the Training**

In [ ]:
import os

# Get API token from environment variable
qrng_token = os.getenv('API_TOKEN')
if qrng_token is None:
    raise ValueError("Please set the API_TOKEN environment variable")

# Train the model
model = train_quantum_gan(qrng_token)

**Important Notes:**

- **API Token**: Ensure your QRNG API token is set in the environment variable `API_TOKEN`.
- **Generated Images**: Check the `generated_images` directory to see the outputs of the generator during training.


# Quantum GAN Results Across Epochs

Below, you can see the progression of images generated by the Quantum GAN model across four different epochs. Observe how the clarity and quality improve over time.

<table>
  <tr>
    <td><strong>Epoch 1</strong></td>
    <td><strong>Epoch 3</strong></td>
    <td><strong>Epoch 22</strong></td>
  </tr>
  <tr>
    <td><img src="./images/fake_images_epoch_0_batch_500.png" width="300" height="300" alt="Epoch 1"></td>
    <td><img src="./images/fake_images_epoch_3_batch_300.png" width="300" height="300" alt="Epoch 3"></td>
    <td><img src="./images/fake_images_epoch_22_batch_100.png" width="300" height="300" alt="Epoch 22"></td>
  </tr>
</table>

<div style="text-align: center;">

<table>
  <tr>
    <td><strong>Epoch 32</strong></td>
  </tr>
  <tr>
    <td><img src="./images/fake_images_epoch_32_batch_300.png" width="300" height="300" alt="Epoch 32"></td>
  </tr>
</table>

</div>

### Progression Details

- **Epoch 1**: Initial results are noisy and pixelated, with some round and circular shapes starting to form.
- **Epoch 3**: Outputs gain more clarity, beginning to form the structure of two distinct digits.
- **Epoch 22**: Clarity is significantly enhanced, with outputs showing much more refined shapes.
- **Epoch 32**: Achieves the best quality overall, with well-defined results and minimal noise.

## **7. Conclusion**

In this tutorial, we've successfully integrated quantum randomness into a GAN for image generation. By using a QRNG, we've introduced true randomness into the model's latent space, potentially enhancing the diversity and unpredictability of the generated images.

**Key Takeaways:**

- **Quantum Randomness**: Provides a source of true randomness, which may improve model robustness.
- **GAN Architecture**: Remains standard, allowing easy integration of quantum noise.
- **Potential Benefits**: Opens avenues for research into the effects of quantum randomness on generative models.

**Next Steps:**

- **Experimentation**: Try training the Quantum GAN on different datasets.
- **Hyperparameter Tuning**: Adjust the latent dimension, learning rates, and network architectures.
- **Comparative Analysis**: Compare the performance of the Quantum GAN with a traditional GAN using PRNGs.

---

## **References**

- **Quantum Random Number Generators**: [Wikipedia](https://en.wikipedia.org/wiki/Quantum_random_number_generator)
- **Generative Adversarial Networks**: [Ian Goodfellow's Paper](https://arxiv.org/abs/1406.2661)
- **PyTorch Documentation**: [PyTorch Official Site](https://pytorch.org/docs/stable/index.html)
- **MNIST Dataset**: [Yann LeCun's Website](http://yann.lecun.com/exdb/mnist/)

---

*Embrace the future of AI by integrating quantum technologies into your models today! If you found this tutorial helpful, please share it with others interested in the exciting intersection of quantum computing and artificial intelligence.*
